### 初始化任务名字（影响数据读取和保存的路径）等

In [ ]:
import torchvision.transforms as transforms
from xy_dataset import XYDataset

TASK = 'demo_1018_2'
width=224
height=224

rootDir = TASK + '/path'
imagesDir= rootDir + '/images'


CATEGORIES = ['apex']

DATASETS = ['A', 'B']

TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

datasets = {}
for name in DATASETS:
    datasets[name] = XYDataset(imagesDir + '_' + name, CATEGORIES, TRANSFORMS, random_hflip=True)

###  显示收集的数据

In [ ]:
import cv2
import ipywidgets
import traitlets
from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget
from imagelist import ImageList
import threading
import time


# 导入记录的图片
imagelist = ImageList(TASK+'/record')

# initialize active dataset
dataset = datasets[DATASETS[0]]


# create image preview
camera_widget = ClickableImageWidget(width=width, height=height)
snapshot_widget = ipywidgets.Image(width=width, height=height)

# create widgets
dataset_widget = ipywidgets.Dropdown(options=DATASETS, description='dataset')
category_widget = ipywidgets.Dropdown(options=CATEGORIES, description='分类')
count_widget = ipywidgets.IntText(description='数量')

# manually update counts at initialization
count_widget.value = dataset.get_count(category_widget.value)

# sets the active dataset
def set_dataset(change):
    global dataset
    dataset = datasets[change['new']]
    count_widget.value = dataset.get_count(category_widget.value)
dataset_widget.observe(set_dataset, names='value')

# update counts when we select a new category
def update_counts(change):
    count_widget.value = dataset.get_count(change['new'])
category_widget.observe(update_counts, names='value')

# 定义播放按钮
playstat = 0
play_button = ipywidgets.Button(description='停止')
def playclick(c):
    global playstat
    if play_button.description == '播放':
        play_button.description = '停止'
        playstat = 0
    else:
        play_button.description = '播放'
        playstat = 1
play_button.on_click(playclick)

forward_button = ipywidgets.Button(description='前进')
def forward(c):
    global imagelist
    imagelist.get_next(step_widget.value)
forward_button.on_click(forward)

back_button = ipywidgets.Button(description='后退')
def back(c):
    global imagelist
    imagelist.get_next(-step_widget.value)
back_button.on_click(back)

step_widget = ipywidgets.IntText(value=1)


play_button.layout.width='15%'
forward_button.layout.width='15%'
step_widget.layout.width='15%'
back_button.layout.width='15%'

def play(camera_widget,play_button):
    global imagelist
    global playstat
    while True :
        time.sleep(0.2)
        if (playstat == 0):
            camera_widget.value = bgr8_to_jpeg(imagelist.get_next());
        else :
            camera_widget.value = bgr8_to_jpeg(imagelist.get_value());
            


execute_thread = threading.Thread(target=play, args=( camera_widget,play_button))
execute_thread.start()


def save_snapshot(_, content, msg):
    if content['event'] == 'click':
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # save to disk
        dataset.save_entry(category_widget.value,  imagelist.get_value(), x, y)
        
        # display saved snapshot
        snapshot = imagelist.get_value().copy()
        snapshot = cv2.circle(snapshot, (x, y), 8, (0, 255, 0), 3)
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        count_widget.value = dataset.get_count(category_widget.value)
        
camera_widget.on_msg(save_snapshot)
camera_collection_widget = ipywidgets.VBox([camera_widget,ipywidgets.HBox([play_button,back_button,step_widget,forward_button])])
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([camera_collection_widget, snapshot_widget]),
    dataset_widget,
    category_widget,
    count_widget
])

display(data_collection_widget)

### 训练

In [ ]:
import torch
import torchvision

device = torch.device('cuda')
output_dim = 2 * len(dataset.categories)  # x, y coordinate for each category

# ALEXNET
# model = torchvision.models.alexnet(pretrained=True)
# model.classifier[-1] = torch.nn.Linear(4096, output_dim)

# SQUEEZENET 
# model = torchvision.models.squeezenet1_1(pretrained=True)
# model.classifier[1] = torch.nn.Conv2d(512, output_dim, kernel_size=1)
# model.num_classes = len(dataset.categories)

# RESNET 18
model = torchvision.models.resnet18(pretrained=True)
model.fc = torch.nn.Linear(512, output_dim)

# RESNET 34
# model = torchvision.models.resnet34(pretrained=True)
# model.fc = torch.nn.Linear(512, output_dim)

# DENSENET 121
# model = torchvision.models.densenet121(pretrained=True)
# model.classifier = torch.nn.Linear(model.num_features, output_dim)

model = model.to(device)

model_save_button = ipywidgets.Button(description='保存模型')
model_load_button = ipywidgets.Button(description='加载模型')
model_path_widget = ipywidgets.Text(description='模型文件', value=rootDir+'/model.pth')

def load_model(c):
    model.load_state_dict(torch.load(model_path_widget.value))
model_load_button.on_click(load_model)
    
def save_model(c):
    torch.save(model.state_dict(), model_path_widget.value)
model_save_button.on_click(save_model)

model_widget = ipywidgets.VBox([
    model_path_widget,
    ipywidgets.HBox([model_load_button, model_save_button])
])


display(model_widget)

### 测试和保存

In [ ]:
from utils import preprocess
import torch.nn.functional as F

state_widget = ipywidgets.ToggleButtons(options=['stop', 'live'], description='state', value='stop')
prediction_widget = ipywidgets.Image(format='jpeg', width=width, height=height)

def live(state_widget, model, imagelist, prediction_widget):
    global dataset
    while state_widget.value == 'live':
        time.sleep(0.1)
        image = imagelist.get_value()
        preprocessed = preprocess(image)
        output = model(preprocessed).detach().cpu().numpy().flatten()
        category_index = dataset.categories.index(category_widget.value)
        x = output[2 * category_index]
        y = output[2 * category_index + 1]
        
        x = int(width * (x / 2.0 + 0.5))
        y = int(height * (y / 2.0 + 0.5))
        
        prediction = image.copy()
        prediction = cv2.circle(prediction, (x, y), 8, (255, 0, 0), 3)
        prediction_widget.value = bgr8_to_jpeg(prediction)
            
def start_live(change):
    if change['new'] == 'live':
        execute_thread = threading.Thread(target=live, args=(state_widget, model, imagelist, prediction_widget))
        execute_thread.start()

state_widget.observe(start_live, names='value')

live_execution_widget = ipywidgets.VBox([
    prediction_widget,
    state_widget
])

#display(live_execution_widget)



BATCH_SIZE = 8

optimizer = torch.optim.Adam(model.parameters())
# optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.9)


epochs_widget = ipywidgets.IntText(description='epochs（次数）', value=1)
eval_button = ipywidgets.Button(description='验证')
train_button = ipywidgets.Button(description='训练')
loss_widget = ipywidgets.FloatText(description='loss')
progress_widget = ipywidgets.FloatProgress(min=0.0, max=1.0, description='进度')


def train_eval(is_training):
    global BATCH_SIZE, LEARNING_RATE, MOMENTUM, model, dataset, optimizer, eval_button, train_button, accuracy_widget, loss_widget, progress_widget, state_widget
    
    try:
        train_loader = torch.utils.data.DataLoader(
            dataset,
            batch_size=BATCH_SIZE,
            shuffle=True
        )

        state_widget.value = 'stop'
        train_button.disabled = True
        eval_button.disabled = True
        time.sleep(1)

        if is_training:
            model = model.train()
        else:
            model = model.eval()

        while epochs_widget.value > 0:
            i = 0
            sum_loss = 0.0
            error_count = 0.0
            for images, category_idx, xy in iter(train_loader):
                # send data to device
                images = images.to(device)
                xy = xy.to(device)

                if is_training:
                    # zero gradients of parameters
                    optimizer.zero_grad()

                # execute model to get outputs
                outputs = model(images)

                # compute MSE loss over x, y coordinates for associated categories
                loss = 0.0
                for batch_idx, cat_idx in enumerate(list(category_idx.flatten())):
                    loss += torch.mean((outputs[batch_idx][2 * cat_idx:2 * cat_idx+2] - xy[batch_idx])**2)
                loss /= len(category_idx)

                if is_training:
                    # run backpropogation to accumulate gradients
                    loss.backward()

                    # step optimizer to adjust parameters
                    optimizer.step()

                # increment progress
                count = len(category_idx.flatten())
                i += count
                sum_loss += float(loss)
                progress_widget.value = i / len(dataset)
                loss_widget.value = sum_loss / i
                
            if is_training:
                epochs_widget.value = epochs_widget.value - 1
            else:
                break
    except e:
        pass
    model = model.eval()

    train_button.disabled = False
    eval_button.disabled = False
    state_widget.value = 'live'
    
train_button.on_click(lambda c: train_eval(is_training=True))
eval_button.on_click(lambda c: train_eval(is_training=False))
    
train_eval_widget = ipywidgets.VBox([
    epochs_widget,
    progress_widget,
    loss_widget,
    ipywidgets.HBox([train_button, eval_button])
])

#display(train_eval_widget)

all_widget = ipywidgets.VBox([
    ipywidgets.HBox([data_collection_widget, live_execution_widget]), 
    train_eval_widget,
    model_widget
])

display(all_widget)